In [1]:
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
from motor_estimation import MotorEstimationSolver
import pandas as pd
import linear_solver
import versor as vsr

In [65]:
import autograd.numpy as np

from pymanopt.manifolds import manifold
from pymanopt.manifolds import Sphere
from pymanopt import Problem
from pymanopt.solvers import SteepestDescent, TrustRegions, ParticleSwarm, ConjugateGradient, NelderMead
from __future__ import division

import numpy as np
import numpy.linalg as la
import numpy.random as rnd

from pymanopt.manifolds.manifold import Manifold

In [66]:
def create_rotor(th_lims=(0, np.pi/2)):
    rotor = vsr.Rot(vsr.Biv(*np.random.uniform(-1, 1, 3)).unit()
                      * np.random.uniform(*th_lims) * -0.5)
    return rotor

In [68]:
class Motor(Manifold):
    def __init__(self):
        self._shape = 8
        self._name = "Motor manifold"

    def __str__(self):
        return self._name

    @property
    def dim(self):
        return 6
    
    

In [13]:
n_points=10

m0 = create_rotor(th_lims=(0,np.pi/3))
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]

In [41]:
def costk(a, b, X): 
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    o0 = a1 * r3 * r3 + (2.0 * a3 * r1 - 2.0 * a2 * r2) * r3 - a1 * r2 * r2 + 2.0 * a3 * r0 * r2 - a1 * r1 * r1 + 2.0 * a2 * r0 * r1 + a1 * r0 * r0 - b1
    o1 = (-a2) * r3 * r3 + (2.0 * a3 * r0 - 2.0 * a1 * r2) * r3 + a2 * r2 * r2 - 2.0 * a3 * r1 * r2 - a2 * r1 * r1 - 2.0 * a1 * r0 * r1 + a2 * r0 * r0 - b2
    o2 = (-a3) * r3 * r3 + (2.0 * a1 * r1 - 2.0 * a2 * r0) * r3 - a3 * r2 * r2 + (-2.0 * a2 * r1 - 2.0 * a1 * r0) * r2 + a3 * r1 * r1 + a3 * r0 * r0 - b3
    return o0 * o0 + o1 * o1 + o2 * o2

def costk2(a, b, X):
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    return ((a3 * b3 + a2 * b2 - a1 * b1) * r3 * r3 +
            ((2.0 * a1 * b2 + 2.0 * a2 * b1) * r2 + 
             (-2.0 * a1 * b3 - 2.0 * a3 * b1) * r1 + 
             (2.0 * a2 * b3 - 2.0 * a3 * b2) * r0) * r3 + 
            (a3 * b3 - a2 * b2 + a1 * b1) * r2 * r2 + 
            ((2.0 * a2 * b3 + 2.0 * a3 * b2) * r1 + 
             (2.0 * a1 * b3 - 2.0 * a3 * b1) * r0) * r2 + 
            ((-a3) * b3 + a2 * b2 + a1 * b1) * r1 * r1 + 
            (2.0 * a1 * b2 - 2.0 * a2 * b1) * r0 * r1 + 
            ((-a3) * b3 - a2 * b2 - a1 * b1) * r0 * r0)

# (2) Define the cost function (here using autograd.numpy)
def cost(X): 
    return np.sum([costk(a, b, X) for a, b in zip(points_a, points_b)])


def cost2(X): 
    return np.sum([costk2(a, b, X) for a, b in zip(points_a, points_b)])

In [42]:
# (1) Instantiate a manifold
manifold = Sphere(4)

In [62]:
problem = Problem(manifold=manifold, cost=cost)

# (3) Instantiate a Pymanopt solver
# solver = SteepestDescent()
solver = TrustRegions()
# solver = ParticleSwarm()
# solver = ConjugateGradient()
# solver = NelderMead()


# let Pymanopt do the rest
Xopt = solver.solve(problem)

print(vsr.Rot(*Xopt))
print(vsr.Rot(*Xopt).rev() * m0)

Compiling cost function...
Computing gradient of cost function...
Computing Hessian of cost function...
Optimizing...
                                            f: +3.687262e+00   |grad|: 1.939151e+01
acc TR+   k:     1     num_inner:     0     f: +2.323287e-01   |grad|: 5.177993e+00   exceeded trust region
acc       k:     2     num_inner:     1     f: +5.099439e-05   |grad|: 8.009882e-02   reached target residual-kappa (linear)
acc       k:     3     num_inner:     1     f: +4.433384e-12   |grad|: 2.516520e-05   reached target residual-theta (superlinear)
acc       k:     4     num_inner:     2     f: +5.610234e-31   |grad|: 1.116276e-15   reached target residual-theta (superlinear)
Terminated - min grad norm reached after 4 iterations, 0.84 seconds.

Rot: [ 0.99 -0.1 -0.033 -0.02 ]
Rot: [ 1 -1.4e-17 1.3e-17 -6.9e-18 ]


In [38]:
problem.backend

In [40]:
import tensorflow as tf
from pymanopt import Problem
from pymanopt.solvers import TrustRegions
from pymanopt.manifolds import Euclidean, Product
# Generate random data
X = np.random.randn(3, 100).astype('float32')
Y = (X[0:1, :] - 2*X[1:2, :] + np.random.randn(1, 100) + 5).astype(
    'float32')

# Cost function is the sqaured test error
w = tf.Variable(tf.zeros([3, 1]))
b = tf.Variable(tf.zeros([1, 1]))
cost = tf.reduce_mean(tf.square(Y - tf.matmul(tf.transpose(w), X) - b))

# first-order, second-order
solver = TrustRegions()

# R^3 x R^1
manifold = Product([Euclidean(3, 1), Euclidean(1, 1)])

# Solve the problem with pymanopt
problem = Problem(manifold=manifold, cost=cost, arg=[w, b], verbosity=0)
wopt = solver.solve(problem)

print('Weights found by pymanopt (top) / '
      'closed form solution (bottom)')

print(wopt[0].T)
print(wopt[1])

X1 = np.concatenate((X, np.ones((1, 100))), axis=0)
wclosed = np.linalg.inv(X1.dot(X1.T)).dot(X1).dot(Y.T)
print(wclosed[0:3].T)
print(wclosed[3])

Weights found by pymanopt (top) / closed form solution (bottom)
[[ 0.92190915 -1.94600466  0.05992358]]
[[ 4.96754547]]
[[ 0.92190918 -1.94600472  0.05992357]]
[ 4.96754568]
